In [ ]:
import numpy as np
import pandas as pd
from transformers import BertModel
from transformers import BertTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertForSequenceClassification

#the virtual environment is new_bert

In [3]:
df=pd.read_csv('../datasets/CREMA-D/MELD-master/data/MELD/train_sent_emo.csv', delimiter=',', nrows = None)


In [4]:
df.head()

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,also I was the point person on my company’s tr...,Chandler,neutral,neutral,0,0,8,21,"00:16:16,059","00:16:21,731"
1,2,You must’ve had your hands full.,The Interviewer,neutral,neutral,0,1,8,21,"00:16:21,940","00:16:23,442"
2,3,That I did. That I did.,Chandler,neutral,neutral,0,2,8,21,"00:16:23,442","00:16:26,389"
3,4,So let’s talk a little bit about your duties.,The Interviewer,neutral,neutral,0,3,8,21,"00:16:26,820","00:16:29,572"
4,5,My duties? All right.,Chandler,surprise,positive,0,4,8,21,"00:16:34,452","00:16:40,917"


In [5]:
text_df=df['Utterance']
text_df=pd.DataFrame(text_df)
text_df.head()


,Utterance
0,also I was the point person on my company’s tr...
1,You must’ve had your hands full.
2,That I did. That I did.
3,So let’s talk a little bit about your duties.
4,My duties? All right.


In [ ]:

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocess(text):
    # Tokenize, pad, and truncate
    return tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors='pt')

text_df['input_ids'] = text_df['Utterance'].apply(lambda x: preprocess(x)['input_ids'][0])
text_df['attention_mask'] = text_df['Utterance'].apply(lambda x: preprocess(x)['attention_mask'][0])


In [ ]:


class TextDataset(Dataset):
    def __init__(self,input_ids,attention_masks):
        self.input_ids=input_ids
        self.attention_masks=attention_masks
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self,idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_masks[idx], dtype=torch.long),
        }

dataset=TextDataset(text_df['input_ids'].to_list(),text_df['attention_mask'].to_list())
train_loader = DataLoader(dataset, shuffle=False)


In [12]:
import torch
print(torch.__version__)  # Check PyTorch version
print(torch.cuda.is_available())  # Check if CUDA is available

2.5.0+cpu
False


In [ ]:

# Load model and force it to use the CPU
model = BertModel.from_pretrained('bert-base-uncased', num_labels=7).to('cpu')
model.eval()

from tqdm import tqdm
encodings = []

# Force DataLoader batches to remain on the CPU
with torch.no_grad():
    for batch in tqdm(train_loader):
        outputs = model(
            input_ids=batch['input_ids'],  # Default is CPU
            attention_mask=batch['attention_mask'],  # Default is CPU
        )
        encoding = outputs.last_hidden_state
        encodings.append(encoding)


  0%|          | 0/9989 [00:00<?, ?it/s]C:\Users\dell\AppData\Local\Temp\ipykernel_18816\707734850.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
C:\Users\dell\AppData\Local\Temp\ipykernel_18816\707734850.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'attention_mask': torch.tensor(self.attention_masks[idx], dtype=torch.long),
100%|██████████| 9989/9989 [29:04<00:00,  5.73it/s]


In [13]:
encodings[9987]


tensor([[[-0.3271, -0.0116, -0.0164,  ..., -0.1454,  0.4163,  0.1177],
         [ 0.7810, -0.1910,  0.3150,  ..., -0.3541,  0.9698, -0.2329],
         [-0.2891, -0.7291,  0.1984,  ...,  0.8826,  0.8325, -0.5647],
         ...,
         [ 0.2801,  0.3044,  0.5951,  ...,  0.1329,  0.4678, -0.0913],
         [-0.2380, -0.1202,  0.5666,  ...,  0.6372,  0.5232, -0.1867],
         [ 0.1859,  0.2200,  0.6492,  ...,  0.0793,  0.3894, -0.0810]]])

In [14]:
text_encodings=[t.flatten() for t in encodings]


In [15]:
bert_encodings_np = [encoding.cpu().numpy() for encoding in text_encodings]


In [16]:
type(bert_encodings_np[0])


numpy.ndarray

In [17]:
import pickle
with open('text_embeddings.pkl', 'wb') as f:
    pickle.dump(bert_encodings_np, f)


In [18]:
with open('text_embeddings.pkl', 'rb') as f:
    loaded_array_list = pickle.load(f)


In [20]:
class TextDataset(Dataset):
    def __init__(self, input_ids, attention_masks, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_masks[idx], dtype=torch.long),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long),
        }

# Prepare labels (map emotion labels to numbers)
label_mapping = {'neutral': 0, 'joy': 1, 'sadness': 2, 'anger': 3, 'surprise': 4, 'fear': 5, 'disgust': 6}
df['label'] = df['Emotion'].map(label_mapping)

# Create dataset and dataloaders
dataset = TextDataset(
    text_df['input_ids'].to_list(),
    text_df['attention_mask'].to_list(),
    df['label'].to_list()
)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)


In [ ]:

# Load pre-trained BERT with classification head
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=7).to('cpu')

# Define optimizer and loss function
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
import os
import torch

def save_checkpoint(epoch, model, optimizer, loss, filepath="checkpoint.pth"):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }
    torch.save(checkpoint, filepath)
    print(f"Checkpoint saved at epoch {epoch + 1}")


In [24]:
epochs = 3
checkpoint_path = "bert_emotion_checkpoint.pth"

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        optimizer.zero_grad()
        outputs = model(
            input_ids=batch['input_ids'].to('cpu'),
            attention_mask=batch['attention_mask'].to('cpu'),
            labels=batch['labels'].to('cpu'),
        )
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    # Save checkpoint at the end of each epoch
    save_checkpoint(epoch, model, optimizer, total_loss / len(train_loader), checkpoint_path)

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}")


C:\Users\dell\AppData\Local\Temp\ipykernel_18816\1479609361.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
C:\Users\dell\AppData\Local\Temp\ipykernel_18816\1479609361.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'attention_mask': torch.tensor(self.attention_masks[idx], dtype=torch.long),


Checkpoint saved at epoch 1
Epoch 1/3, Loss: 1.137200417610022
Checkpoint saved at epoch 2
Epoch 2/3, Loss: 0.9028418294537943
Checkpoint saved at epoch 3
Epoch 3/3, Loss: 0.6272407133167925


In [ ]:
#####################################


#testing the model 


####################################

In [26]:
import pandas as pd
from transformers import BertTokenizer

# Load the test dataset
test_df = pd.read_csv('../datasets/CREMA-D/MELD-master/data/MELD/test_sent_emo.csv', delimiter=',')

# Preprocess the text data (like you did for training data)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocess(text):
    return tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors='pt')

test_df['input_ids'] = test_df['Utterance'].apply(lambda x: preprocess(x)['input_ids'][0])
test_df['attention_mask'] = test_df['Utterance'].apply(lambda x: preprocess(x)['attention_mask'][0])


In [27]:
# Map emotions to numbers (same as training)
label_mapping = {'neutral': 0, 'joy': 1, 'sadness': 2, 'anger': 3, 'surprise': 4, 'fear': 5, 'disgust': 6}
test_df['label'] = test_df['Emotion'].map(label_mapping)


In [ ]:
################################

#audio part :


###############################

In [ ]:
import os

# Path to the audio directory in MELD dataset
audio_dir = "../datasets/MELD/audio"

# List all audio files
audio_files = os.listdir(audio_dir)

# Check the number of audio files
print(f"Number of audio files: {len(audio_files)}")

# Display the first 10 files
print("First 10 audio files:")
print(audio_files[:10])
